In [1]:
import open_clip

/Users/juansegundohevia/anaconda3/envs/elec542project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch 

In [3]:
torch.set_default_device("mps")

In [18]:

MODEL_NAME = 'ViT-B-32'
model, _, preprocess = open_clip.create_model_and_transforms(MODEL_NAME, pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer(MODEL_NAME)


In [7]:
sample_text = "A photograph of a field"
_text_emb = tokenizer([sample_text])
_text_emb.size()

torch.Size([1, 77])

In [8]:
sample_text = "A photograph of a field in a cornfield"
_text_emb = tokenizer([sample_text])
_text_emb.size()

torch.Size([1, 77])

In [3]:
%cd ..

/Users/juansegundohevia/Documents/repos/ELEC542-TextGuidedRecolorization


In [4]:
import importlib

In [55]:
from model.palette_encoder import PaletteEncoder
importlib.reload(PaletteEncoder)
enc = PaletteEncoder.TextCLIPEncoder(in_channels=512, n_blocks=3).to("mps")

In [61]:
enc(["this is as sky"])

torch.Size([512])


tensor([-0.0127, -0.0059, -0.0160, -0.0048,  0.1023,  0.0751,  0.0147, -0.0622,
        -0.0086,  0.0240, -0.0695,  0.0546, -0.0877,  0.0671, -0.0115,  0.0811,
         0.0316,  0.0315,  0.0164, -0.0337,  0.0827,  0.0138, -0.0485, -0.0343,
        -0.0640, -0.0276,  0.0377, -0.0028, -0.0204, -0.0519, -0.0186, -0.0009,
        -0.0250, -0.0581, -0.0753,  0.0268,  0.0506,  0.0010,  0.0307,  0.0745,
         0.0604, -0.0780, -0.0675,  0.0776,  0.0354,  0.0146,  0.0298,  0.0419,
        -0.0201,  0.0668, -0.0331,  0.0632, -0.0589, -0.0068, -0.0205,  0.0175,
        -0.0204, -0.0804,  0.0721,  0.0500, -0.1049, -0.0026, -0.1008,  0.0604],
       device='mps:0', grad_fn=<LinearBackward0>)

In [56]:
from model.palette_encoder.PaletteEncoder import PaletteDecoder

In [75]:
dec = PaletteDecoder(in_channels=64, n_blocks=3).to("mps")

In [73]:
v = enc(["this is as sky"])
v.size()

torch.Size([512])


torch.Size([64])

In [72]:
v.unsqueeze(0).size()

torch.Size([1, 64])

In [77]:
dec(v.unsqueeze(0)).size()

torch.Size([1, 15])

# data loader

In [11]:
import data_loader as d

In [28]:
importlib.reload(d)

<module 'data_loader' from '/Users/juansegundohevia/Documents/repos/ELEC542-TextGuidedRecolorization/data_loader.py'>

In [29]:
ds = d.Text2PaletteDataset("data/hexcolor_vf/train_names.pkl",
                         "data/hexcolor_vf/train_palettes_rgb.pkl",
                         50)

In [7]:
import pickle as pkl 
# try opening the names pkl file
with open("data/hexcolor_vf/train_names.pkl", "rb") as f:
    names = pkl.load(f)

In [23]:
tokenizer(" ".join(names[0]))

tensor([[49406,   585,   533,  3153, 49407,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]])

In [30]:
ds[0]

(tensor([49406.,   585.,   533.,  3153., 49407.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.]),
 array([[0.3372549 , 0.51372549, 0.75294118],
        [0.59215686, 0.86666667, 0.81960784],
        [0.50196078, 0.42745098, 0.83137255],
        [0.62745098, 0.81568627, 0.85490196],
        [0.74901961, 0.65490196, 0.89803922]]))

In [33]:
# test trained decoder
from model.palette_encoder import PaletteEncoder as t2p

dec = t2p.PaletteDecoder(n_blocks=4,
                           in_channels=77, 
                           num_colors=5,
                           reduce_factor=2).to("mps")
checkpoint = torch.load("models/PaletteEncoder/model_epoch_6750.pth", map_location="mps")
dec.load_state_dict(checkpoint)

/var/folders/2c/s1dc7gqn0jz4dls2kprq822m0000gn/T/ipykernel_75215/2441792748.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("models/PaletteEncode

<All keys matched successfully>

In [37]:

MODEL_NAME = 'ViT-B-32'
model, _, preprocess = open_clip.create_model_and_transforms(MODEL_NAME, pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer(MODEL_NAME)


In [41]:
# test decoder
dec.eval()
v = tokenizer(["happy day in the woods"])
v.shape
v = model.encode_text(v).to("mps")
with torch.no_grad():
    output = dec(v)

RuntimeError: linear(): input and weight.T shapes cannot be multiplied (1x512 and 77x77)

In [43]:
v.size()

torch.Size([1, 512])